In [3]:
# the access key
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

In [4]:
# Code to get access token 

import requests
import base64

def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token


In [5]:
def get_genres(access_token, limit=50, offset=0):
    genres_url = 'https://api.spotify.com/v1/browse/categories'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'limit': limit,
        'offset': offset
    }
    response = requests.get(genres_url, headers=headers, params=params)
    genres_data = response.json()
    genres = [(category['id'], category['name']) for category in genres_data['categories']['items']]
    return genres


In [6]:
access_token = get_access_token(client_id, client_secret)  
genres = get_genres(access_token)  

print("List of genres:")
genre_list = []
for genre_id, genre_name in genres:
    genre_list.append(genre_name)

print(genre_list)


List of genres:
['Made For You', 'New Releases', 'Summer', 'Country', 'Hip-Hop', 'Pop', 'Charts', 'GLOW', 'Black Music Month', 'Latin', 'Student', 'Discover', 'Rock', 'Dance/Electronic', 'In the car', 'Mood', 'Indie', 'R&B', 'Christian & Gospel', 'Workout', 'Disney', 'Música Mexicana', 'K-pop', 'Chill', 'Netflix', 'Sleep', 'Party', 'At Home', 'Decades', 'Love', 'Metal', 'Spotify CLASSICS', 'Jazz', 'Trending', 'Frequency', 'Classical', 'Folk & Acoustic', 'Focus', 'Soul', 'Kids & Family', 'Gaming', 'Anime', 'TV & Movies', 'Instrumental', 'Punk', 'Ambient', 'Blues', 'Cooking & Dining', 'Alternative', 'Travel']


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_genres(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']

access_token = get_access_token(client_id, client_secret)  
def get_artist_genre(artist_id):
    artist = sp.artists(artist_id)
    return artist['genres']

In [18]:
import pandas as pd
import time

def create_dataframe(query_list, limit=50):  
    
    data = {
        'Track ID': [],
        'Track name': [],
        'Artist name': [],
        'Popularity score': [],
        'Release year': [],
        'Genre': []
    }
    
    for query in query_list:
        
        search_results = sp.search(query, limit=limit, type='track')
        tracks = search_results['tracks']['items']
        
        
        for track in tracks: 
            data['Track ID'].append(track['id'])
            data['Track name'].append(track['name'])
            data['Artist name'].append(track['artists'][0]['name'])
            data['Popularity score'].append(track['popularity'])
            data['Release year'].append(track['album']['release_date'][:4])
            data['Genre'].append(query)

                
    df = pd.DataFrame(data)
    return df


# List of genres
genres = [
    'Made For You', 'New Releases', 'Summer', 'Country', 'Hip-Hop', 'Pop', 'Charts', 'GLOW', 'Black Music Month', 
    'Latin', 'Student', 'Discover', 'Rock', 'Dance/Electronic', 'In the car', 'Mood', 'Indie', 'R&B', 
    'Christian & Gospel', 'Workout', 'Disney', 'Música Mexicana', 'K-pop', 'Chill', 'Netflix', 'Sleep', 
    'Party', 'At Home', 'Decades', 'Love', 'Metal', 'Spotify CLASSICS', 'Jazz', 'Trending', 'Frequency', 
    'Classical', 'Folk & Acoustic', 'Focus', 'Soul', 'Kids & Family', 'Gaming', 'Anime', 'TV & Movies', 
    'Instrumental', 'Punk', 'Ambient', 'Blues', 'Cooking & Dining', 'Alternative', 'Travel'
]

# Create the DataFrame with 50 tracks from each genre
spotify_df = create_dataframe(genres, limit=50) 


In [19]:
from IPython.display import display
display(spotify_df)

spotify_df.to_csv('spotify.csv', index=False)


,Track ID,Track name,Artist name,Popularity score,Release year,Genre
0,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,80,1979,Made For You
1,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,79,2023,Made For You
2,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,66,2019,Made For You
3,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,71,2016,Made For You
4,55ikcenXPbQdzCnO1sOuYg,I Was Made For Lovin’ You - from The Fall Guy,YUNGBLUD,72,2024,Made For You
...,...,...,...,...,...,...
2495,1HpBm13TNcyMW3PoUrlZPv,But Anyway,Blues Traveler,52,1990,Travel
2496,3rhXtYLlWXNCC5sjXkj6dc,Travelin' Man,Dick Curless,34,1966,Travel
2497,0fvQvActgYOEihSxVPrqNP,The Traveling Song,will.i.am,46,2008,Travel
2498,0wVyUtOvYWJvniPRidlNnQ,White Noise Flakes,The White Noise Travelers,64,2022,Travel


In [10]:
spotify_df["Genre"].value_counts()

Genre
Made For You          50
Focus                 50
At Home               50
Decades               50
Love                  50
Metal                 50
Spotify CLASSICS      50
Jazz                  50
Trending              50
Frequency             50
Classical             50
Folk & Acoustic       50
Soul                  50
New Releases          50
Kids & Family         50
Gaming                50
Anime                 50
TV & Movies           50
Instrumental          50
Punk                  50
Ambient               50
Blues                 50
Cooking & Dining      50
Alternative           50
Party                 50
Sleep                 50
Netflix               50
Chill                 50
Summer                50
Country               50
Hip-Hop               50
Pop                   50
Charts                50
GLOW                  50
Black Music Month     50
Latin                 50
Student               50
Discover              50
Rock                  50
Dance/Electronic   

In [11]:
def get_audio_features(track_ids):
    features_list = []  

    for i in range(0, len(track_ids), 50):  
        batch = track_ids[i:i+50] 
        features_results = sp.audio_features(batch)  
        features_list += features_results  

    return features_list


In [12]:
track_ids = spotify_df['Track ID'].tolist()
audio_features = get_audio_features(track_ids)


In [13]:

audio_features = [feature for feature in audio_features if feature is not None]


audio_features_df = pd.DataFrame(audio_features)
merged_df = spotify_df.merge(audio_features_df, left_on='Track ID', right_on='id', how='inner')


In [14]:
merged_df

,Track ID,Track name,Artist name,Popularity score,Release year,Genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1uRQzm3P0nRGnSfnuzMTpq,Made For You,OneRepublic,35,2009,Made For You,0.6400,0.555,7,-10.204,...,0.0981,0.3950,122.038,audio_features,1uRQzm3P0nRGnSfnuzMTpq,spotify:track:1uRQzm3P0nRGnSfnuzMTpq,https://api.spotify.com/v1/tracks/1uRQzm3P0nRG...,https://api.spotify.com/v1/audio-analysis/1uRQ...,257493,4
1,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,80,1979,Made For You,0.7680,0.852,4,-6.215,...,0.0723,0.8670,128.373,audio_features,07q0QVgO56EorrSGHC48y3,spotify:track:07q0QVgO56EorrSGHC48y3,https://api.spotify.com/v1/tracks/07q0QVgO56Eo...,https://api.spotify.com/v1/audio-analysis/07q0...,271240,4
2,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,78,2023,Made For You,0.7960,0.468,7,-6.375,...,0.1180,0.0677,119.980,audio_features,0WIv5qV41y6YjjB9V1biuC,spotify:track:0WIv5qV41y6YjjB9V1biuC,https://api.spotify.com/v1/tracks/0WIv5qV41y6Y...,https://api.spotify.com/v1/audio-analysis/0WIv...,188000,4
3,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,66,2019,Made For You,0.5810,0.441,1,-6.829,...,0.1110,0.3370,82.125,audio_features,7vF3xkCMvZjAe2nTWY0uQZ,spotify:track:7vF3xkCMvZjAe2nTWY0uQZ,https://api.spotify.com/v1/tracks/7vF3xkCMvZjA...,https://api.spotify.com/v1/audio-analysis/7vF3...,238400,4
4,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,70,2016,Made For You,0.6120,0.799,10,-4.603,...,0.1720,0.6870,125.976,audio_features,7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zVCrzzEJU7u24sbJPXA5W,https://api.spotify.com/v1/tracks/7zVCrzzEJU7u...,https://api.spotify.com/v1/audio-analysis/7zVC...,188253,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,2duHuwtyCt5hoRjg6Ijoee,Travelin' Man - Live In Detroit/1975,Bob Seger,40,1976,Travel,0.3900,0.898,2,-4.006,...,0.0905,0.1860,119.714,audio_features,2duHuwtyCt5hoRjg6Ijoee,spotify:track:2duHuwtyCt5hoRjg6Ijoee,https://api.spotify.com/v1/tracks/2duHuwtyCt5h...,https://api.spotify.com/v1/audio-analysis/2duH...,280173,4
2541,0wVyUtOvYWJvniPRidlNnQ,White Noise Flakes,The White Noise Travelers,63,2022,Travel,0.1570,0.228,10,-32.985,...,0.1120,0.0349,83.378,audio_features,0wVyUtOvYWJvniPRidlNnQ,spotify:track:0wVyUtOvYWJvniPRidlNnQ,https://api.spotify.com/v1/tracks/0wVyUtOvYWJv...,https://api.spotify.com/v1/audio-analysis/0wVy...,272000,4
2542,4pxj2nEkmypTXihGBv6gB1,Travelin' Shoes,Elvin Bishop,31,1974,Travel,0.5740,0.897,5,-6.230,...,0.0357,0.9100,119.954,audio_features,4pxj2nEkmypTXihGBv6gB1,spotify:track:4pxj2nEkmypTXihGBv6gB1,https://api.spotify.com/v1/tracks/4pxj2nEkmypT...,https://api.spotify.com/v1/audio-analysis/4pxj...,440173,4
2543,5aRuOoUKNGYdzeJYM4s3mL,Travelin’ Soldier,Bubba Ward,30,2022,Travel,0.5510,0.512,8,-8.975,...,0.0826,0.4680,72.851,audio_features,5aRuOoUKNGYdzeJYM4s3mL,spotify:track:5aRuOoUKNGYdzeJYM4s3mL,https://api.spotify.com/v1/tracks/5aRuOoUKNGYd...,https://api.spotify.com/v1/audio-analysis/5aRu...,275278,4


In [15]:
unique_df = merged_df.drop_duplicates(subset='Track ID', keep='first')

unique_df

,Track ID,Track name,Artist name,Popularity score,Release year,Genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1uRQzm3P0nRGnSfnuzMTpq,Made For You,OneRepublic,35,2009,Made For You,0.6400,0.555,7,-10.204,...,0.0981,0.3950,122.038,audio_features,1uRQzm3P0nRGnSfnuzMTpq,spotify:track:1uRQzm3P0nRGnSfnuzMTpq,https://api.spotify.com/v1/tracks/1uRQzm3P0nRG...,https://api.spotify.com/v1/audio-analysis/1uRQ...,257493,4
1,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,80,1979,Made For You,0.7680,0.852,4,-6.215,...,0.0723,0.8670,128.373,audio_features,07q0QVgO56EorrSGHC48y3,spotify:track:07q0QVgO56EorrSGHC48y3,https://api.spotify.com/v1/tracks/07q0QVgO56Eo...,https://api.spotify.com/v1/audio-analysis/07q0...,271240,4
2,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,78,2023,Made For You,0.7960,0.468,7,-6.375,...,0.1180,0.0677,119.980,audio_features,0WIv5qV41y6YjjB9V1biuC,spotify:track:0WIv5qV41y6YjjB9V1biuC,https://api.spotify.com/v1/tracks/0WIv5qV41y6Y...,https://api.spotify.com/v1/audio-analysis/0WIv...,188000,4
3,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,66,2019,Made For You,0.5810,0.441,1,-6.829,...,0.1110,0.3370,82.125,audio_features,7vF3xkCMvZjAe2nTWY0uQZ,spotify:track:7vF3xkCMvZjAe2nTWY0uQZ,https://api.spotify.com/v1/tracks/7vF3xkCMvZjA...,https://api.spotify.com/v1/audio-analysis/7vF3...,238400,4
4,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,70,2016,Made For You,0.6120,0.799,10,-4.603,...,0.1720,0.6870,125.976,audio_features,7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zVCrzzEJU7u24sbJPXA5W,https://api.spotify.com/v1/tracks/7zVCrzzEJU7u...,https://api.spotify.com/v1/audio-analysis/7zVC...,188253,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,2duHuwtyCt5hoRjg6Ijoee,Travelin' Man - Live In Detroit/1975,Bob Seger,40,1976,Travel,0.3900,0.898,2,-4.006,...,0.0905,0.1860,119.714,audio_features,2duHuwtyCt5hoRjg6Ijoee,spotify:track:2duHuwtyCt5hoRjg6Ijoee,https://api.spotify.com/v1/tracks/2duHuwtyCt5h...,https://api.spotify.com/v1/audio-analysis/2duH...,280173,4
2541,0wVyUtOvYWJvniPRidlNnQ,White Noise Flakes,The White Noise Travelers,63,2022,Travel,0.1570,0.228,10,-32.985,...,0.1120,0.0349,83.378,audio_features,0wVyUtOvYWJvniPRidlNnQ,spotify:track:0wVyUtOvYWJvniPRidlNnQ,https://api.spotify.com/v1/tracks/0wVyUtOvYWJv...,https://api.spotify.com/v1/audio-analysis/0wVy...,272000,4
2542,4pxj2nEkmypTXihGBv6gB1,Travelin' Shoes,Elvin Bishop,31,1974,Travel,0.5740,0.897,5,-6.230,...,0.0357,0.9100,119.954,audio_features,4pxj2nEkmypTXihGBv6gB1,spotify:track:4pxj2nEkmypTXihGBv6gB1,https://api.spotify.com/v1/tracks/4pxj2nEkmypT...,https://api.spotify.com/v1/audio-analysis/4pxj...,440173,4
2543,5aRuOoUKNGYdzeJYM4s3mL,Travelin’ Soldier,Bubba Ward,30,2022,Travel,0.5510,0.512,8,-8.975,...,0.0826,0.4680,72.851,audio_features,5aRuOoUKNGYdzeJYM4s3mL,spotify:track:5aRuOoUKNGYdzeJYM4s3mL,https://api.spotify.com/v1/tracks/5aRuOoUKNGYd...,https://api.spotify.com/v1/audio-analysis/5aRu...,275278,4


In [16]:
import numpy as np
import pandas 

columns_to_drop = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms']
cleaned_spotify = unique_df.drop(columns=columns_to_drop)
cleaned_spotify.columns

Index(['Track ID', 'Track name', 'Artist name', 'Popularity score',
       'Release year', 'Genre', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')

In [17]:
cleaned_spotify.to_csv('final.csv', index=False)